In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
reconstructed_model = tf.keras.models.load_model('final_model.h5')

In [3]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import train_test_split
import json
import re
import tensorflow as tf
import numpy as np
import more_itertools

In [4]:
candidate = pd.read_excel("candidate_master_data.xlsx",sheet_name='Candidate Data')
candidate.to_csv("candidate_master.csv",index=None,header=True)
cand_df = pd.read_csv("candidate_master.csv")

In [5]:
def skill_parsing(skill):
    '''
    Function that extracts the skills from the Skill column 
    '''
    res = json.loads(skill,strict=False)
    skill_list=res['skills']
    
    skills = []
    
    for i in range (len(skill_list)):
        if skill_list[i]['skill_type'] == 'primary':
            skills.append(skill_list[i]['skill'])
        else:
            if skill_list[i]['skill_type'] == 'secondary':
                skills.append(skill_list[i]['skill'])
    return skills

can_skills ={}
skill_dict = dict(zip(cand_df['Candidate ID'],cand_df['Skills']))

for i in range (len(skill_dict)):
    for k,v in skill_dict.items():
        values=skill_dict[k]
        skill_set=skill_parsing(values)
        can_skills.update({k:skill_set})

In [6]:
requisition = pd.read_excel("requisition_data.xlsx",sheet_name='Requisition Data')
requisition.to_csv("requisition.csv",index=None,header=True)
req_df = pd.read_csv("requisition.csv")
req_df = req_df[req_df['Skills'].notna()]

In [7]:
final_df=cand_df[['Candidate ID','Current Designation','Total Experience ( in Months)', 'Education__degree']]
final_df.drop_duplicates('Candidate ID',inplace=True)

C:\Users\gopinath.selvaraj\Anaconda3\envs\env\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [8]:
can_keys=can_skills.keys()
can_values=can_skills.values()

In [9]:
new_list=[]
str1=' '
for i in range (len(can_values)):
    s=str1.join(list(can_values)[i])
    new_list.append(s)
    str1=' '

data={'Candidate ID':can_keys, 'Can_Skills':new_list}
skill_df = pd.DataFrame.from_dict(data)

In [10]:
final_df=final_df.merge(skill_df,left_on=['Candidate ID'], right_on=['Candidate ID'],how='left')

In [11]:
# Check for columns which arent numbers
for label, content in final_df.items():
    if not pd.api.types.is_numeric_dtype(content):
        print(label)

Current Designation
Education__degree
Can_Skills


In [12]:
# Turn categorical variables into numbers and fill them
for label, content in final_df.items():
    if not pd.api.types.is_numeric_dtype(content):
        final_df[label] = pd.Categorical(content).codes + 1

In [13]:
# Find the columns that contains strings
for label, content in final_df.items():
    if pd.api.types.is_string_dtype(content):
        print(label)

In [14]:
# This will change all string values to categories
for label, content in final_df.items():
    if pd.api.types.is_string_dtype(content):
        final_df[label] = content.astype("category").cat.as_ordered()

In [16]:
X = final_df[['Candidate ID','Current Designation','Total Experience ( in Months)','Education__degree','Can_Skills']]

In [17]:
X_test = X.drop(['Candidate ID'],axis=1)

In [18]:
from sklearn.preprocessing import MinMaxScaler
scalar = MinMaxScaler()
scalar.fit(X_test)
X_trans = scalar.transform(X_test)

In [19]:
pred_values = reconstructed_model.predict(X_trans)

In [22]:
y_score = []
for i in range (len(pred_values)):
    y_score.append(pred_values[i][0])

In [23]:
final_score=dict(zip(X['Candidate ID'],y_score))

In [24]:
data1 = {'Candidate ID':final_score.keys(), 'Score':final_score.values()}
score_dataframe = pd.DataFrame.from_dict(data1)

In [25]:
score_dataframe['Rank']=score_dataframe['Score'].rank(method='max',ascending=False)
score_dataframe=score_dataframe.sort_values(by='Rank')
score_dataframe

,Candidate ID,Score,Rank
86,1001641355,2.955318,1.0
96,1001668286,2.259871,2.0
46,1001458569,2.064599,3.0
0,1000114023,1.959308,4.0
64,1001512025,1.770907,5.0
...,...,...,...
37,1001442691,0.000000,107.0
74,1001561725,0.000000,107.0
75,1001597493,0.000000,107.0
10,1000667374,0.000000,107.0
